In [1]:
import xspec
import numpy as np
import os
import glob2

In [2]:
os.chdir('../data/')

In [6]:
source_nums1 = np.loadtxt('Interested_sources1.txt', dtype='object')

In [10]:
source_nums2 = np.loadtxt('Interested_sources2.txt', dtype='object')

In [31]:
source_nums3 = np.loadtxt('../jupyter_nbs/Interested_sources3.txt', dtype='object')

In [7]:
source_nums1

array(['204003401010013', '208251401010018', '201129715010042',
       '202026705010034', '207641901010009', '201129707010033',
       '204003401010007', '208016803010003', '201129704010033',
       '204003401010008', '201129721010126', '201129721010090',
       '208016813010003', '201113501010154', '201129707010012',
       '201129701010027', '201129719010007', '201129721010057',
       '202039301010054', '208016803010001', '201129707010063',
       '201129705010038', '201129701010011', '208251401010006',
       '201129719010010', '201129721010060', '201129705010011',
       '201129721010044', '201129705010079', '201129702010002',
       '201129721010003', '201129706010014'], dtype=object)

In [16]:
source_nums2

array(['208251401010018', '201129715010042', '202026705010034',
       '208251401010019', '207641901010009', '201129707010033',
       '201129707010002', '204003401010007', '208016803010003',
       '205062912010039', '201129704010033', '201113501010148',
       '204003401010008', '201129721010126', '201129721010090',
       '208016813010003', '201113501010154', '201129707010012',
       '201113501010066', '201129701010027', '201129719010007',
       '201129721010057', '202039301010054', '208016803010001',
       '201129707010063', '201129705010038', '201129701010011',
       '204105804010005', '201129705010015', '206946404015068',
       '201129719010010', '201129721010138', '201129721010060',
       '201129705010011', '201129705010079', '201129702010002',
       '201129721010003', '201129706010014'], dtype=object)

In [34]:
source_nums3

array(['208251401010016', '204003401010013', '208251401010018',
       '202039301010071', '201129715010042', '201113501010122',
       '202026705010034', '201129702010029', '207641901010009',
       '201129707010033', '201129702010049', '201129704015082',
       '204003401010007', '208016803010003', '201129707010048',
       '201129704010033', '201113501010148', '204003401010008',
       '201129721010126', '208016813010003', '201129721010130',
       '202026708010224', '201129704010027', '201129707010012',
       '201129701010027', '201113501010169', '201129721010057',
       '202039301010054', '208016803010001', '201129707010063',
       '206556701010012', '201129705010038', '201129701010011',
       '201129704010030', '208251401010006', '201113501010035',
       '201129705015062', '201129719010010', '201129721010138',
       '201129721010060', '201129705010011', '201129721010044',
       '201129705010079', '201129702010002', '201129721010003'],
      dtype=object)

In [37]:
extra_sources3 = []
for source in source_nums3:
    if not source in source_nums1:
        extra_sources3.append(source)

In [38]:
extra_sources3

['208251401010016',
 '202039301010071',
 '201113501010122',
 '201129702010029',
 '201129702010049',
 '201129704015082',
 '201129707010048',
 '201113501010148',
 '201129721010130',
 '202026708010224',
 '201129704010027',
 '201113501010169',
 '206556701010012',
 '201129704010030',
 '201113501010035',
 '201129705015062',
 '201129721010138']

In [15]:
extra_sources2

['204003401010013', '208251401010006', '201129721010044']

In [3]:
def fit_src_pl(source_num):
    spec_pnfile = glob2.glob('./Galactic_highNH_combinedXMM/' +
                             str(source_num) + '_PN_combined_src_grp1*')
    print(spec_pnfile)
    spec_mosfile = glob2.glob('./Galactic_highNH_combinedXMM/' +
                              str(source_num) + '_MOS_combined_src_grp1*')
    print(spec_mosfile)
    xspec.plot.xaxis = 'keV'
    xspec.AllData.clear()
    if len(spec_pnfile) > 0:
        spec_pn = xspec.Spectrum(spec_pnfile[0])
        spec_pn.ignore('0.0-2.0, 10.0-**')
    if len(spec_mosfile) > 0:
        spec_mos = xspec.Spectrum(spec_mosfile[0])
        spec_mos.ignore('0.0-2.0, 10.0-**')
    model = xspec.Model('tbabs*pegpwrlw')
    model.setPars({1:10, 2:1.5, 3:2.0, 4:10.0})
    xspec.Fit.statMethod = 'cstat'
    xspec.Fit.statTest = 'cvm'
    xspec.Fit.renorm()
    xspec.Fit.perform()
    xspec.Fit.error('2.706 2')
    xspec.Fit.perform()
    goodness = xspec.Fit.goodness()
    print('Model params for ' + source_num +
          ': NH = ', xspec.AllModels(1).TBabs.nH.values[0],
          ' Gamma: ', xspec.AllModels(1).pegpwrlw.PhoIndex.values[0],
          ' norm: ', xspec.AllModels(1).pegpwrlw.norm.values[0])
    print('Fit stats for ' + source_num + ': ', xspec.Fit.statistic,
          xspec.Fit.testStatistic, goodness, xspec.Fit.dof)
    print('\n')
    return ([source_num, xspec.Fit.statistic, xspec.Fit.testStatistic, goodness,
             xspec.Fit.dof],
            [source_num, xspec.AllModels(1).TBabs.nH.values[0],
             xspec.AllModels(1).pegpwrlw.PhoIndex.values[0],
             xspec.AllModels(1).pegpwrlw.norm.values[0]])

In [21]:
source_nums[11]

'201129721010090'

In [22]:
fit_src_pl(source_nums[0])

Model params for 204003401010013: NH =  5.570948783187327  Gamma:  0.2856837748882508  norm:  0.09594845190330294
Fit stats for 204003401010013:  967.4081109810052 -7.127565163657311 47.0 1167




(['204003401010013', 967.4081109810052, -7.127565163657311, 47.0, 1167],
 ['204003401010013',
  5.570948783187327,
  0.2856837748882508,
  0.09594845190330294])

In [22]:
xspec.Plot.device = '/xw'
xspec.AllData.clear()

In [25]:
os.getcwd()

'/Users/pavanrh/Documents/UofA_projects/GalacticBulge_Xrayclassify/data'

In [21]:
fit_stats_table = []
model_params_table = []

In [26]:
for source_num in extra_sources:
    print(source_num)
    fit_stats, model_params = fit_src_pl(source_num)
    fit_stats_table.append(fit_stats)
    model_params_table.append(model_params)

208251401010019
['./Galactic_highNH_combinedXMM/208251401010019_PN_combined_src_grp1_969cts.ds']
['./Galactic_highNH_combinedXMM/208251401010019_MOS_combined_src_grp1_171cts.ds']
Model params for 208251401010019: NH =  6.6758256742059165  Gamma:  0.6259714924729392  norm:  0.31197104852555724
Fit stats for 208251401010019:  510.44131734675943 -7.198223400084045 74.0 618


201129707010002
['./Galactic_highNH_combinedXMM/201129707010002_PN_combined_src_grp1_746cts.ds']
['./Galactic_highNH_combinedXMM/201129707010002_MOS_combined_src_grp1_103cts.ds']
Model params for 201129707010002: NH =  10.156874937049176  Gamma:  0.5318688781997305  norm:  0.21246815461664836
Fit stats for 201129707010002:  528.1984538608845 -5.604083210058599 100.0 559


205062912010039
['./Galactic_highNH_combinedXMM/205062912010039_PN_combined_src_grp1_1248cts.ds']
[]
Model params for 205062912010039: NH =  13.65025373546186  Gamma:  1.2042848471707344  norm:  0.12096078369785197
Fit stats for 205062912010039:  597

In [32]:
for source_num in source_nums:
    fit_stats, model_params = fit_src_pl(source_num)
    fit_stats_table.append(fit_stats)
    model_params_table.append(model_params)

Model params for 204003401010013: NH =  5.569084939523726  Gamma:  0.2856805082089663  norm:  0.0959397787940848
Fit stats for 204003401010013:  967.4081187928718 -7.127891721802487 54.0 1167


Model params for 208251401010018: NH =  5.162514096354813  Gamma:  0.07036555663328947  norm:  0.12035130182863246
Fit stats for 208251401010018:  605.8682437676152 -7.2861073197369635 63.0 664


Model params for 201129715010042: NH =  22.457804446025378  Gamma:  1.5167394570156585  norm:  0.19414522422658076
Fit stats for 201129715010042:  485.0866593201038 -4.8372406355445605 100.0 557


Model params for 202026705010034: NH =  7.021031505281972  Gamma:  2.52118068959353  norm:  0.0915545809991841
Fit stats for 202026705010034:  1255.178738757072 -8.669731165977511 63.0 1404


Model params for 207641901010009: NH =  10.902718007241548  Gamma:  0.5057393348589655  norm:  0.335679286177105
Fit stats for 207641901010009:  403.4875697401384 -6.873088101949593 83.0 428


Model params for 20112970701

In [16]:
from astropy.table import Table

In [29]:
Table(np.array(model_params_table))

col0,col1,col2,col3
str32,str32,str32,str32
204003401010013,5.569084939523726,0.2856805082089663,0.0959397787940848
208251401010018,5.162514096354813,0.07036555663328947,0.12035130182863246


In [9]:
def fit_src_pl_gauss(source_num):
    spec_pnfile = glob2.glob('./Galactic_highNH_combinedXMM/' +
                             str(source_num) + '_PN_combined_src_grp1*')
    spec_mosfile = glob2.glob('./Galactic_highNH_combinedXMM/' +
                              str(source_num) + '_MOS_combined_src_grp1*')
    num_files = len(spec_pnfile) + len(spec_mosfile)
    xspec.plot.xaxis = 'keV'
    xspec.AllData.clear()
    if len(spec_pnfile) > 0:
        spec_pn = xspec.Spectrum(spec_pnfile[0])
        spec_pn.ignore('0.0-2.0, 10.0-**')
    if len(spec_mosfile) > 0:
        spec_mos = xspec.Spectrum(spec_mosfile[0])
        spec_mos.ignore('0.0-2.0, 10.0-**')
    
    model = xspec.Model('tbabs*(pegpwrlw+gauss)')
    model.TBabs.nH.values = [10, 0.001, 0.1, 0.1, 500, 500]
    model.pegpwrlw.PhoIndex.values = [1.0, 0.001, -3, -3, 5, 5]
    model.pegpwrlw.eMin.values = '2.0'
    model.pegpwrlw.eMax.values = '10.0'
    model.gaussian.LineE.values = '6.7, 0.0001, 5.0, 5.0, 8.0, 8.0'
    model.gaussian.Sigma.values = '0.1 0'
    model.gaussian.norm.values = '1.0E-5, 0.001,,,,'
    
    xspec.Fit.statMethod = 'cstat'
    xspec.Fit.statTest = 'cvm'
    xspec.Fit.renorm()
    xspec.Fit.query = 'yes'
    xspec.Fit.perform()
    xspec.Fit.error('2.706 6')
    xspec.Fit.perform()
    xspec.Fit.error('2.706 8')
    xspec.Fit.perform()
    xspec.Fit.error('2.706 6')
    xspec.Fit.error('2.706 8')
    goodness = xspec.Fit.goodness(1000)
    line_pos_low = xspec.AllModels(1).gaussian.LineE.error[0]
    line_pos_high = xspec.AllModels(1).gaussian.LineE.error[1]
    line_norm_low = xspec.AllModels(1).gaussian.norm.error[0]
    line_norm_high = xspec.AllModels(1).gaussian.norm.error[1]
    print('Model params for ' + source_num +
          ': NH = ', xspec.AllModels(1).TBabs.nH.values[0],
          ' Gamma: ', xspec.AllModels(1).pegpwrlw.PhoIndex.values[0],
          ' norm: ', xspec.AllModels(1).pegpwrlw.norm.values[0],
          ' Fe_pos:', xspec.AllModels(1).gaussian.LineE.values[0],
          line_pos_low, line_pos_high,
          ' Fe_norm:', xspec.AllModels(1).gaussian.norm.values[0],
          line_norm_low, line_norm_high)
    print('Fit stats for ' + source_num + ': ', xspec.Fit.statistic,
          xspec.Fit.testStatistic, goodness, xspec.Fit.dof)
    print('\n')
    return ([source_num, xspec.Fit.statistic, xspec.Fit.testStatistic, goodness,
             xspec.Fit.dof],
            [source_num, xspec.AllModels(1).TBabs.nH.values[0],
             xspec.AllModels(1).pegpwrlw.PhoIndex.values[0],
             xspec.AllModels(1).pegpwrlw.norm.values[0],
             xspec.AllModels(1).gaussian.LineE.values[0], line_pos_low,
             line_pos_high,
             xspec.AllModels(1).gaussian.norm.values[0], line_norm_low,
             line_norm_high])

In [ ]:
fit_src_pl_gauss(source_nums[8])

In [11]:
xspec.AllData.clear()

In [28]:
for source_num in extra_sources:
    fit_stats, model_params = fit_src_pl_gauss(source_num)
    fit_stats_table.append(fit_stats)
    model_params_table.append(model_params)

Model params for 208251401010019: NH =  6.993863241843736  Gamma:  0.8886846362935485  norm:  0.2845572993727834  Fe_pos: 6.416098313853779 6.296800192825516 6.545557138956404  Fe_norm: 1.645702016968644e-06 6.765882763606243e-07 2.788788008280213e-06
Fit stats for 208251401010019:  501.60829744710657 -7.296890452723536 68.4 616


Model params for 201129707010002: NH =  9.03326769815316  Gamma:  0.5317661609320895  norm:  0.1904525316931469  Fe_pos: 6.475879802412365 6.370291019748035 6.586141042033478  Fe_norm: 8.873644865342694e-07 3.8831768992449163e-07 1.4495665107626946e-06
Fit stats for 201129707010002:  518.635691917013 -5.616959438804859 100.0 557


Model params for 205062912010039: NH =  9.657274940248525  Gamma:  1.0472095664779155  norm:  0.08936032686582175  Fe_pos: 6.406688294882384 6.349132701668823 6.464454656676042  Fe_norm: 1.0506310894086828e-06 6.80788825006108e-07 1.4775582937480066e-06
Fit stats for 205062912010039:  570.9943523335717 -7.744898357008618 81.2 699




In [15]:
from astropy.table import Table
from astropy.table.pprint import conf
conf.max_lines = -1
conf.max_width = -1

In [16]:
Table(np.array(fit_stats_table))

col0,col1,col2,col3,col4
str32,str32,str32,str32,str32
204003401010013,948.6046980145245,-7.152019162228451,50.8,1165
208251401010018,598.1950965644372,-7.301386640595909,66.6,662
201129715010042,481.7588495908445,-4.870359335273696,100.0,555
202026705010034,1253.5975056313011,-8.502110078367632,80.2,1402
207641901010009,389.9355842573409,-6.887963426487899,81.3,426
201129707010033,401.07543969973466,-8.435543669268187,49.4,474
204003401010007,1140.1696725507222,-8.145519162320365,35.5,1385
208016803010003,626.698373509101,-6.642897135170608,96.5,771
201129704010033,662.1728184878098,-7.119892330184164,64.6,757


In [25]:
Table(np.array(model_params_table))

col0,col1,col2,col3,col4,col5,col6,col7,col8,col9
str32,str32,str32,str32,str32,str32,str32,str32,str32,str32
204003401010013,6.492184383628125,0.829273232098666,0.4602495211780489,6.6790512653352305,6.618447714431509,6.73594063299563,8.491633845006755e-07,4.91610701446695e-07,1.2726130580479031e-06
208251401010018,5.6345405810303095,0.33775648482674864,0.4625666672447684,6.740535547056095,6.633493147038876,6.841369601395712,5.726648261021955e-07,2.1480061018509981e-07,9.87332785214769e-07
201129715010042,21.287336787440523,1.5513007825855791,0.4908925681799981,6.808123571381358,6.638505364100071,6.987502658508773,3.950317231540448e-07,3.785018909902422e-08,8.042740669216307e-07
202026705010034,6.904221104202331,2.485826039850627,2.873133408640566,8.0,0.0,0.0,5.39613945180951e-24,0.8224962005991274,0.0
207641901010009,9.933661454783781,0.673109472762957,0.6013312835593951,6.5063994491433785,6.420559285079521,6.592401726736457,1.9078377800198045e-06,9.747859072444128e-07,3.02005468561944e-06
201129707010033,9.221828674985119,0.3302942641599104,0.5140637072882585,6.740671465500665,5.968581026142564,6.941754868751346,4.618635199094286e-07,4.721088056810407e-08,9.400743968619199e-07
204003401010007,4.205263237215881,1.0570321775129892,0.36350579809698746,7.553839941704776,0.0,0.0,3.424083809633005e-07,0.0,7.566849912391498e-07
208016803010003,4.341944600168478,1.3586722029741647,0.6762179983935588,6.260121157856911,0.0,0.0,6.341253074003629e-07,0.0,1.4530663221567019e-06
201129704010033,23.417020459425455,1.0847885270821505,0.462184631552291,6.697512811442056,6.451217920410059,6.922433659321229,5.96142730808614e-07,7.635118918872073e-08,1.1901371947154166e-06


In [17]:
os.getcwd()

'/Users/pavanrh/Documents/UofA_projects/GalacticBulge_Xrayclassify/data'

In [29]:
fit_stats_table

[['208251401010019', 510.44131734675943, -7.198223400084045, 74.0, 618],
 ['201129707010002', 528.1984538608845, -5.604083210058599, 100.0, 559],
 ['205062912010039', 597.5521267154752, -6.344992861221887, 100.0, 701],
 ['201113501010148', 1081.9804889200107, 0.21035987671302828, 100.0, 1140],
 ['201113501010066', 888.2556175345751, -8.207198336266908, 43.0, 1100],
 ['204105804010005', 1497.7378569744433, -6.489639520726686, 100.0, 1834],
 ['201129705010015', 1343.0744444435293, -5.368913801581472, 100.0, 1607],
 ['206946404015068', 1677.9845795701503, -5.701157939232663, 100.0, 1703],
 ['201129721010138', 2773.4053155099527, -6.585117500948999, 100.0, 2880],
 ['208251401010019', 501.60829744710657, -7.296890452723536, 68.4, 616],
 ['201129707010002', 518.635691917013, -5.616959438804859, 100.0, 557],
 ['205062912010039', 570.9943523335717, -7.744898357008618, 81.2, 699],
 ['201113501010148', 1077.7139681908493, 0.18815342909552443, 100.0, 1138],
 ['201113501010066', 881.6878182273861,

In [39]:
fit_stats_table = []
model_params_table = []

In [42]:
for source_num in extra_sources3:
    fit_stats, model_params = fit_src_pl_gauss(source_num)
    fit_stats_table.append(fit_stats)
    model_params_table.append(model_params)

Model params for 208251401010016: NH =  3.315991553970805  Gamma:  -0.2137150285422311  norm:  0.11599336221074164  Fe_pos: 6.898150669475832 6.815885678301465 6.984487527557104  Fe_norm: 7.269257276604836e-07 3.237155938621042e-07 1.199239438325992e-06
Fit stats for 208251401010016:  493.33690955895486 -7.089883941595301 50 644


Model params for 202039301010071: NH =  0.39064057076349923  Gamma:  -1.2019102454542008  norm:  0.07480139571166076  Fe_pos: 6.566704218284519 6.131856196781664 6.813966096004029  Fe_norm: 2.797119921029875e-07 7.703027600216289e-08 5.046882481318894e-07
Fit stats for 202039301010071:  594.1506003223099 -7.4958830717773175 50 689


Model params for 201113501010122: NH =  4.760226132859209  Gamma:  0.8312471821731653  norm:  0.2549010208839435  Fe_pos: 6.679250745018266 6.628028476128676 6.728492285381931  Fe_norm: 3.0854691805551297e-06 2.0886481506091736e-06 4.219490302327192e-06
Fit stats for 201113501010122:  570.8098685422134 -5.905298868147503 50 611




In [43]:
for source_num in extra_sources3:
    fit_stats, model_params = fit_src_pl(source_num)
    fit_stats_table.append(fit_stats)
    model_params_table.append(model_params)

['./Galactic_highNH_combinedXMM/208251401010016_PN_combined_src_grp1_873cts.ds']
['./Galactic_highNH_combinedXMM/208251401010016_MOS_combined_src_grp1_187cts.ds']
Model params for 208251401010016: NH =  2.9354208055676057  Gamma:  -0.4716169261978168  norm:  0.13022435968455665
Fit stats for 208251401010016:  503.4237519807334 -6.903241405441625 50 646


['./Galactic_highNH_combinedXMM/202039301010071_PN_combined_src_grp1_1123cts.ds']
['./Galactic_highNH_combinedXMM/202039301010071_MOS_combined_src_grp1_128cts.ds']
Model params for 202039301010071: NH =  27.137083654820568  Gamma:  0.2145160944244535  norm:  0.15013033342814364
Fit stats for 202039301010071:  596.8154933679252 -5.849411505282365 50 691


['./Galactic_highNH_combinedXMM/201113501010122_PN_combined_src_grp1_725cts.ds']
['./Galactic_highNH_combinedXMM/201113501010122_MOS_combined_src_grp1_167cts.ds']
Model params for 201113501010122: NH =  4.391838659691195  Gamma:  0.4515750203508291  norm:  0.29727572521853934
Fit stats

In [4]:
bright_sources = np.loadtxt('BrightSources.txt', dtype=object)

In [6]:
len(bright_sources)

184

In [10]:
bright_fit_stats_pl_gauss = []
bright_model_params_pl_gauss = []
for source_num in bright_sources:
    fit_stats, model_params = fit_src_pl_gauss(source_num)
    bright_fit_stats_pl_gauss.append(fit_stats)
    bright_model_params_pl_gauss.append(model_params)

Model params for 208251401010016: NH =  3.315991553970805  Gamma:  -0.213715028542231  norm:  0.11599336221074164  Fe_pos: 6.898150669475834 6.815885678301465 6.984487527557104  Fe_norm: 7.269257276604839e-07 3.237155938621042e-07 1.199239438325992e-06
Fit stats for 208251401010016:  493.3369095589546 -7.089883941595308 72.4 644


Model params for 201129705010043: NH =  34.67209142778243  Gamma:  3.0588579923850325  norm:  1.1576684808884872  Fe_pos: 6.557662265047674 6.421135725202022 6.7026097200687085  Fe_norm: 4.223798296873443e-06 2.200289131522451e-06 7.414113456078438e-06
Fit stats for 201129705010043:  741.3953410665504 -5.78769630683624 94.5 896


Model params for 205056701010026: NH =  20.368160214324835  Gamma:  1.0564155778304465  norm:  0.36756879769979534  Fe_pos: 6.416971967150056 6.3998020617255245 6.4348143547419445  Fe_norm: 5.560837669927268e-06 4.709502967027004e-06 6.58998119093035e-06
Fit stats for 205056701010026:  1303.5271374630106 -6.012450747177178 100.0 1421

Model params for 201129705010038: NH =  36.388221401965886  Gamma:  2.241512163170961  norm:  0.6817050608187163  Fe_pos: 6.835672492540434 6.684845427605658 6.99950804129211  Fe_norm: 1.1161479675326628e-06 3.473105407880592e-07 1.983149813422442e-06
Fit stats for 201129705010038:  1744.6641623745627 -6.034411722360908 98.4 2074


Model params for 202039301010071: NH =  0.39064057076349856  Gamma:  -1.2019102454542008  norm:  0.07480139571166076  Fe_pos: 6.566704218284519 6.131856196781664 6.813984379572198  Fe_norm: 2.797119921029875e-07 7.703027600216289e-08 5.046882481318894e-07
Fit stats for 202039301010071:  594.1506003223091 -7.4958830717773175 67.8 689


Model params for 201129715010042: NH =  21.250977837560775  Gamma:  1.5466773519772705  norm:  0.17944342390077825  Fe_pos: 6.808588889355971 6.638393866752943 6.987137999215485  Fe_norm: 3.9456815255596907e-07 3.693031239378537e-08 8.074719444128345e-07
Fit stats for 201129715010042:  481.7588495908445 -4.870359335273696 100.0

Model params for 202026707010114: NH =  2.483146896626881  Gamma:  0.07968108127944498  norm:  0.06944652730863145  Fe_pos: 6.581135185700155 6.48090845447981 6.686888276311542  Fe_norm: 4.959970441337675e-07 3.1259669849394427e-07 6.915013835678597e-07
Fit stats for 202026707010114:  1545.7824554776546 -5.624465524836741 99.9 1761


Model params for 201129721010138: NH =  3.7242367571526613  Gamma:  0.7375003581735685  norm:  0.1524804143405937  Fe_pos: 6.789458203378014 6.3600577728731205 6.940841494639607  Fe_norm: 2.541433014919318e-07 7.037507510576207e-08 4.465495752206252e-07
Fit stats for 201129721010138:  2768.128804986469 -6.587037425928399 100.0 2878


Model params for 201129706010006: NH =  8.813269894647345  Gamma:  3.0922463142050867  norm:  1.7537219263569832  Fe_pos: 6.641727770202937 6.638640667884733 6.646402907123408  Fe_norm: 1.3145981101460148e-05 1.280997555540464e-05 1.3488381785679446e-05
Fit stats for 201129706010006:  6089.5306342882795 -7.616932258037508 100.

Model params for 201129704010014: NH =  8.916949659637224  Gamma:  3.1058257643067466  norm:  1.3436991657253252  Fe_pos: 6.6475563112352 6.638309721607408 6.659464634584663  Fe_norm: 1.1133730142097561e-05 1.0349828449329117e-05 1.1937002105861944e-05
Fit stats for 201129704010014:  2923.4610556160333 -7.798365643605015 100.0 2628


Model params for 201129707010017: NH =  17.00051849148541  Gamma:  0.6897435541045788  norm:  0.3453273106136597  Fe_pos: 6.83494295474467 6.766903554582764 6.909161362001742  Fe_norm: 1.8658464841476932e-06 1.3011072049183438e-06 2.481180987866582e-06
Fit stats for 201129707010017:  967.4710126857709 -8.772462542850185 62.3 1056


Model params for 201129701010011: NH =  4.739995082011656  Gamma:  1.192852581336838  norm:  0.13435443298221997  Fe_pos: 7.059179900546556 6.851883440462578 7.2468685769357135  Fe_norm: 2.6031106926612827e-07 1.6679036059572633e-08 5.317486220029556e-07
Fit stats for 201129701010011:  1528.7536265871954 -6.903780687651161 99.7 

Model params for 201129704010045: NH =  6.1510524850274395  Gamma:  0.06489413573624586  norm:  0.17372455968705594  Fe_pos: 6.741507303021312 6.586585562556312 6.986637627106757  Fe_norm: 9.675868311417766e-07 4.5305439784307753e-07 1.5606003466423202e-06
Fit stats for 201129704010045:  648.7417930239742 -7.226320548136451 78.1 747


Model params for 201129702010049: NH =  2.8732027771000896  Gamma:  -0.8654160901623011  norm:  0.08240700002194243  Fe_pos: 6.796614035776706 6.6573582679453045 6.94671437400279  Fe_norm: 3.989488093680435e-07 1.1055488832257242e-07 7.296544410753731e-07
Fit stats for 201129702010049:  457.0486455526801 -8.024583711024606 41.6 633


Model params for 206946406010093: NH =  10.553773180747928  Gamma:  0.16396387005789387  norm:  0.13928798007732507  Fe_pos: 6.687558147890736 6.551053487758277 6.836657904176575  Fe_norm: 6.994974111316063e-07 2.630028785038725e-07 1.223868615382958e-06
Fit stats for 206946406010093:  444.81342130642554 -7.621311800288128 48

Model params for 205062912010039: NH =  9.657274940248525  Gamma:  1.0472095664779155  norm:  0.08936032686582175  Fe_pos: 6.406688294882384 6.349132701668823 6.464454656676042  Fe_norm: 1.0506310894086828e-06 6.80788825006108e-07 1.4775582937480066e-06
Fit stats for 205062912010039:  570.9943523335717 -7.744898357008618 75.8 699


Model params for 202039301010017: NH =  11.448107458365117  Gamma:  2.0183807128385136  norm:  0.09368232238481025  Fe_pos: 6.673425796476647 6.641900254302197 6.706475934121258  Fe_norm: 1.4610448978288006e-06 1.1607363123651258e-06 1.8022857489384268e-06
Fit stats for 202039301010017:  692.2726431710901 -6.402829010450521 98.9 894


Model params for 201129707010006: NH =  8.345731957734163  Gamma:  0.49047355287953925  norm:  0.22404422108538638  Fe_pos: 6.5089172227041 6.442575683416225 6.569386419723647  Fe_norm: 1.525500172486073e-06 1.0819671286899587e-06 2.0002101643767035e-06
Fit stats for 201129707010006:  975.7822119659143 -6.414999485808918 100.0 

Model params for 201129721010027: NH =  19.28449403904466  Gamma:  1.5491843215913756  norm:  0.38432116324582716  Fe_pos: 6.670447834390839 6.637050691174819 6.702792114372811  Fe_norm: 1.6999387272695278e-06 1.446421889394655e-06 1.9620403714711706e-06
Fit stats for 201129721010027:  3104.9052883429777 -6.307351936438739 100.0 2977


Model params for 201129705015062: NH =  6.324481325998436  Gamma:  0.4658811821769544  norm:  0.08280445364443656  Fe_pos: 6.695910795117269 6.594864475098071 6.798368481393229  Fe_norm: 4.170397214529866e-07 2.307783724393165e-07 6.158837442969484e-07
Fit stats for 201129705015062:  2455.267232541939 -2.7790234184113327 100.0 2467


Model params for 201113501010101: NH =  50.06726795554267  Gamma:  2.392805610548896  norm:  0.5626389739654882  Fe_pos: 7.09144514310312 6.932073800426782 7.231514467710556  Fe_norm: 1.9827347499707923e-06 5.905167797481301e-07 8.358027300208115e-06
Fit stats for 201113501010101:  851.3478244700748 -4.734851723421842 96.1 1

Model params for 201129721010060: NH =  8.384141681372023  Gamma:  1.4364300939794175  norm:  0.15211446285413793  Fe_pos: 5.0 0.0 5.043596480970597  Fe_norm: 2.1989556408050633e-07 8.45880407013138e-08 3.590387869278208e-07
Fit stats for 201129721010060:  2247.344139310151 -9.094985749705442 83.3 2437


Model params for 201129721010090: NH =  4.553180198485429  Gamma:  -0.11801420027445425  norm:  0.10004444868798755  Fe_pos: 6.475517237418866 6.380455447660737 6.570912602207281  Fe_norm: 8.403576611000175e-07 4.2150884906902416e-07 1.3270134081110777e-06
Fit stats for 201129721010090:  560.5454012774072 -8.174352566171143 31.6 726


Model params for 201129705010060: NH =  2.6474656337698477  Gamma:  0.01652547197608909  norm:  0.14761013322149988  Fe_pos: 6.673451714656477 6.642295413756265 6.70994907942882  Fe_norm: 9.809261858189386e-07 8.415884839299459e-07 1.1252376804929674e-06
Fit stats for 201129705010060:  3068.7735206272705 -7.986603634936724 99.9 3012


Model params for 201

In [7]:
bright_fit_stats_pl = []
bright_model_params_pl = []
for source_num in bright_sources:
    fit_stats, model_params = fit_src_pl(source_num)
    bright_fit_stats_pl.append(fit_stats)
    bright_model_params_pl.append(model_params)

['./Galactic_highNH_combinedXMM/208251401010016_PN_combined_src_grp1_873cts.ds']
['./Galactic_highNH_combinedXMM/208251401010016_MOS_combined_src_grp1_187cts.ds']
Model params for 208251401010016: NH =  2.9353702954360874  Gamma:  -0.47162511602769847  norm:  0.13022431307503113
Fit stats for 208251401010016:  503.4237463804997 -6.903238753548963 77.0 646


['./Galactic_highNH_combinedXMM/201129705010043_PN_combined_src_grp1_1161cts.ds']
['./Galactic_highNH_combinedXMM/201129705010043_MOS_combined_src_grp1_488cts.ds']
Model params for 201129705010043: NH =  40.33033523007585  Gamma:  2.8604198999776624  norm:  1.4710677168910191
Fit stats for 201129705010043:  756.0545447348501 -5.751403659532808 96.0 898


['./Galactic_highNH_combinedXMM/205056701010026_PN_combined_src_grp1_2648cts.ds']
['./Galactic_highNH_combinedXMM/205056701010026_MOS_combined_src_grp1_502cts.ds']
Model params for 205056701010026: NH =  54.019097354178164  Gamma:  2.483798192705852  norm:  1.734407608698709
Fit sta

Model params for 208251401010018: NH =  5.162514050417679  Gamma:  0.0703655732774187  norm:  0.12035130183722652
Fit stats for 208251401010018:  605.8682437675683 -7.286107307326092 71.0 664


['./Galactic_highNH_combinedXMM/201129721010009_PN_combined_src_grp1_9103cts.ds']
['./Galactic_highNH_combinedXMM/201129721010009_MOS_combined_src_grp1_5567cts.ds']
Model params for 201129721010009: NH =  8.78580066161392  Gamma:  3.021462018071625  norm:  0.8075338591805623
Fit stats for 201129721010009:  2693.1954155354465 -7.318860322685598 100.0 2552


['./Galactic_highNH_combinedXMM/201129705010038_PN_combined_src_grp1_3637cts.ds']
['./Galactic_highNH_combinedXMM/201129705010038_MOS_combined_src_grp1_2126cts.ds']
Model params for 201129705010038: NH =  37.38421501140375  Gamma:  2.096691665804689  norm:  0.7052409922850986
Fit stats for 201129705010038:  1750.5203428709303 -5.99153131719591 100.0 2076


['./Galactic_highNH_combinedXMM/202039301010071_PN_combined_src_grp1_1123cts.ds']
['./Ga

Model params for 201129705015041: NH =  3.282071231018731  Gamma:  0.1718959288675245  norm:  0.2345844886591158
Fit stats for 201129705015041:  3373.005619840601 -5.517360735094185 100.0 3002


['./Galactic_highNH_combinedXMM/201129721010133_PN_combined_src_grp1_2553cts.ds']
[]
Model params for 201129721010133: NH =  0.17256342924312126  Gamma:  -0.46157996856495803  norm:  0.13837656733527154
Fit stats for 201129721010133:  855.7485288717667 -9.220370504899815 8.0 1035


['./Galactic_highNH_combinedXMM/201129721010081_PN_combined_src_grp1_5667cts.ds']
['./Galactic_highNH_combinedXMM/201129721010081_MOS_combined_src_grp1_1809cts.ds']
Model params for 201129721010081: NH =  45.91867221626882  Gamma:  1.4236329282820204  norm:  0.35874255590212073
Fit stats for 201129721010081:  2104.4131485913617 -5.187467149971367 100.0 2312


['./Galactic_highNH_combinedXMM/201113503010004_PN_combined_src_grp1_451835cts.ds']
['./Galactic_highNH_combinedXMM/201113503010004_MOS_combined_src_grp1_463009

Model params for 201113501010092: NH =  53.90066167973326  Gamma:  1.3374796132100288  norm:  0.4943291269244425
Fit stats for 201113501010092:  2722.157550977552 -5.170401754923747 100.0 2764


['./Galactic_highNH_combinedXMM/201129702010029_PN_combined_src_grp1_1258cts.ds']
['./Galactic_highNH_combinedXMM/201129702010029_MOS_combined_src_grp1_241cts.ds']
Model params for 201129702010029: NH =  48.60383562599405  Gamma:  1.489733210430539  norm:  0.2977791089207776
Fit stats for 201129702010029:  704.455676506788 -6.116537452198342 70.0 789


['./Galactic_highNH_combinedXMM/204024304010028_PN_combined_src_grp1_2122cts.ds']
['./Galactic_highNH_combinedXMM/204024304010028_MOS_combined_src_grp1_630cts.ds']
Model params for 204024304010028: NH =  93.8494075200078  Gamma:  2.6050921874816146  norm:  4.7994237753514435
Fit stats for 204024304010028:  1327.4157452944435 -4.455340965988229 100.0 1361


['./Galactic_highNH_combinedXMM/201129706010008_PN_combined_src_grp1_1864cts.ds']
['./Galac

Model params for 201129704010019: NH =  16.189732920246918  Gamma:  0.5114151328561345  norm:  0.2131539022858881
Fit stats for 201129704010019:  1029.6902173410692 -8.694217069773899 22.0 1218


['./Galactic_highNH_combinedXMM/201129715010038_PN_combined_src_grp1_859cts.ds']
[]
Model params for 201129715010038: NH =  21.538831625114984  Gamma:  0.35047604729001647  norm:  0.2449704484324465
Fit stats for 201129715010038:  448.0335312962698 -6.4113758883098395 100.0 527


['./Galactic_highNH_combinedXMM/201113501010010_PN_combined_src_grp1_785960cts.ds']
['./Galactic_highNH_combinedXMM/201113501010010_MOS_combined_src_grp1_439478cts.ds']
Model params for 201113501010010: NH =  20.011978881012258  Gamma:  2.355574052232925  norm:  67.3512850711179
Fit stats for 201113501010010:  31446.051884329267 -6.362843490917895 100.0 3195


['./Galactic_highNH_combinedXMM/201129704010030_PN_combined_src_grp1_1693cts.ds']
['./Galactic_highNH_combinedXMM/201129704010030_MOS_combined_src_grp1_787cts.d

Model params for 201129721010022: NH =  27.659170268272593  Gamma:  1.539810130493413  norm:  0.594363237514793
Fit stats for 201129721010022:  2380.7548067475445 -4.318089928757167 100.0 2392


['./Galactic_highNH_combinedXMM/206946408010020_PN_combined_src_grp1_1098cts.ds']
[]
Model params for 206946408010020: NH =  2.8495415335614562  Gamma:  -0.07824992342584847  norm:  0.08293790030975273
Fit stats for 206946408010020:  483.12213036311397 -6.991658613071714 97.0 616


['./Galactic_highNH_combinedXMM/204105804010005_PN_combined_src_grp1_1643cts.ds']
['./Galactic_highNH_combinedXMM/204105804010005_MOS_combined_src_grp1_2662cts.ds']
Model params for 204105804010005: NH =  10.483895052575194  Gamma:  1.926137860037752  norm:  0.4985943113015151
Fit stats for 204105804010005:  1497.7378569744433 -6.489639520726686 100.0 1834


['./Galactic_highNH_combinedXMM/208016803010003_PN_combined_src_grp1_868cts.ds']
['./Galactic_highNH_combinedXMM/208016803010003_MOS_combined_src_grp1_341cts.ds'

Model params for 201129721010017: NH =  9.353771589543248  Gamma:  1.477635957498021  norm:  0.3488199855555787
Fit stats for 201129721010017:  2491.4660941581797 -7.902473905252832 100.0 2500


['./Galactic_highNH_combinedXMM/206904418010089_PN_combined_src_grp1_986cts.ds']
[]
Model params for 206904418010089: NH =  93.04674629573869  Gamma:  2.405513057723319  norm:  2.969531494972342
Fit stats for 206904418010089:  616.939700223494 -5.6405468387876665 94.0 578


['./Galactic_highNH_combinedXMM/201129721010063_PN_combined_src_grp1_4446cts.ds']
[]
Model params for 201129721010063: NH =  7.150473022475801  Gamma:  1.492741276794224  norm:  0.13446532717067927
Fit stats for 201129721010063:  1167.7560383311675 -8.747499725631204 89.0 1306


['./Galactic_highNH_combinedXMM/205547504010002_PN_combined_src_grp1_16726cts.ds']
['./Galactic_highNH_combinedXMM/205547504010002_MOS_combined_src_grp1_16921cts.ds']
Model params for 205547504010002: NH =  9.762733585902478  Gamma:  1.76921661867941

Model params for 203032102010002: NH =  9.696896047548847  Gamma:  1.4808840582460294  norm:  13.756811719404292
Fit stats for 203032102010002:  9196.357885070807 -4.562575269178919 100.0 3164


['./Galactic_highNH_combinedXMM/201129705010067_PN_combined_src_grp1_6958cts.ds']
['./Galactic_highNH_combinedXMM/201129705010067_MOS_combined_src_grp1_2355cts.ds']
Model params for 201129705010067: NH =  22.467989899660214  Gamma:  1.4636831644347197  norm:  0.26479538245506035
Fit stats for 201129705010067:  2181.793226399197 -3.4799253548033513 100.0 2426


['./Galactic_highNH_combinedXMM/208016803010001_PN_combined_src_grp1_1144cts.ds']
['./Galactic_highNH_combinedXMM/208016803010001_MOS_combined_src_grp1_723cts.ds']
Model params for 208016803010001: NH =  3.4546720237740045  Gamma:  0.7110039170545263  norm:  0.631816793139819
Fit stats for 208016803010001:  991.6005526434735 -7.286961214195922 98.0 1133


['./Galactic_highNH_combinedXMM/201129721010023_PN_combined_src_grp1_3031cts.ds']
['